In [ ]:
import os
import datetime
from pathlib import Path
import glob
import itertools

folder = r"C:\Users\feder\OneDrive\Documents\Slovenija\Kemijski_Institut\Code\Image Alignment\LUMICKS_image_alignment\TIRF IRM"
transform_matrix = r"transform_matrix.json"
timestamps = []
tiff_filelist = glob.glob(f"{folder}/*.tiff")
# output_path = os.path.join(folder, "aligned_output")
output_path = "aligned_output"
print(output_path)

def roundTime(dt=None, roundTo=60):
   """Round a datetime object to any time lapse in seconds
   dt : datetime.datetime object, default now.
   roundTo : Closest number of seconds to round to, default 1 minute.
   Author: Thierry Husson 2012 - Use it as you want but don't blame me.
   """
   if dt == None : dt = datetime.datetime.now()
   seconds = (dt.replace(tzinfo=None) - dt.min).seconds
   rounding = (seconds+roundTo/2) // roundTo * roundTo
   return dt + datetime.timedelta(0,rounding-seconds,-dt.microsecond)

def get_timestamp(filename):
    round_to_seconds=30
    time = roundTime(datetime.datetime.fromtimestamp(os.path.getmtime(filename)),round_to_seconds)
    print(time)
    return time


tiff_filelist = sorted(tiff_filelist, key=get_timestamp)
wt_irm = itertools.groupby(tiff_filelist, get_timestamp)

for key, group in wt_irm:
    group = list(group)
    if len(group) == 2:
        if "_IRM" in group[0]:
            irm_file = group[0]
            wt_file = group[1]
        else:
            irm_file = group[1]
            wt_file = group[0]
        assert "_IRM" in irm_file, "no IRM file"
        assert "_WT" in wt_file, "no WT file"
        cmd_string = f'python image_aligner.py "{wt_file}" "{irm_file}" -m "{transform_matrix}" -o {output_path}'
        print(cmd_string)
        os.system(cmd_string)
    else:
        if len(group) > 2:
            raise f"{len(group)} files were found in the same group"


""" for file in tiff_filelist:
    time = datetime.datetime.fromtimestamp(os.path.getmtime(file)).replace(
        second=0, microsecond=0
    )
    if time not in timestamps:
        timestamps.append(time)

for timestamp in timestamps:
    for file in tiff_filelist:
        time = datetime.datetime.fromtimestamp(os.path.getmtime(file)).replace(
            second=0, microsecond=0
        )
        if time == timestamp:
            if "WT" in file:
                wt_file = file
            else:
                irm_file = file 

    output_path = os.path.join(folder, Path(wt_file).stem + "_output")

    print(f"!python image_aligner.py {wt_file} {irm_file} -m '{transform_matrix}' -o {output_path}") """

aligned_output
2023-12-07 14:21:11
2023-12-07 14:22:20
2023-12-07 14:21:03
2023-12-07 14:22:15
2023-12-12 13:49:20
2023-12-12 13:50:40
2023-12-12 13:49:20
2023-12-12 13:50:41
2023-12-07 14:21:03
2023-12-07 14:21:11
2023-12-07 14:22:15
2023-12-07 14:22:20
2023-12-12 13:49:20
2023-12-12 13:49:20
2023-12-12 13:50:40
python image_aligner.py "C:\Users\feder\OneDrive\Documents\Slovenija\Kemijski_Institut\Code\Image Alignment\LUMICKS_image_alignment\TIRF IRM\20231212-134920_WT_B1.tiff" "C:\Users\feder\OneDrive\Documents\Slovenija\Kemijski_Institut\Code\Image Alignment\LUMICKS_image_alignment\TIRF IRM\20231212-134007_IRM_B1.tiff" -m "transform_matrix.json" -o aligned_output
2023-12-12 13:50:41


' for file in tiff_filelist:\n    time = datetime.datetime.fromtimestamp(os.path.getmtime(file)).replace(\n        second=0, microsecond=0\n    )\n    if time not in timestamps:\n        timestamps.append(time)\n\nfor timestamp in timestamps:\n    for file in tiff_filelist:\n        time = datetime.datetime.fromtimestamp(os.path.getmtime(file)).replace(\n            second=0, microsecond=0\n        )\n        if time == timestamp:\n            if "WT" in file:\n                wt_file = file\n            else:\n                irm_file = file \n\n    output_path = os.path.join(folder, Path(wt_file).stem + "_output")\n\n    print(f"!python image_aligner.py {wt_file} {irm_file} -m \'{transform_matrix}\' -o {output_path}") '

In [3]:
!python image_aligner.py --help

usage: image_aligner.py [-h] [-o OUTPUT_DIRECTORY] [-m TRANSFORM_MATRIX]
                        [-f FIT_METHOD] [-b BOX_SIZE] [-g MIN_GRADIENT]
                        [-e MAX_POS_ERROR] [-p MAX_PHOTONS]
                        [-d DELETE_TEMP_FILES]
                        wt_file irm_file

Scripts to align various channels based on reference beads.

positional arguments:
  wt_file               WT tif file
  irm_file              IRM tif file

options:
  -h, --help            show this help message and exit
  -o OUTPUT_DIRECTORY, --output-directory OUTPUT_DIRECTORY
                        Output directory. Default=output/
  -m TRANSFORM_MATRIX, --transform-matrix TRANSFORM_MATRIX
                        Previously calculated matrix in .json format
  -f FIT_METHOD, --fit-method FIT_METHOD
                        Fit method for picasso. Default=lq
  -b BOX_SIZE, --box-size BOX_SIZE
                        Box sized for picasso. Default=21
  -g MIN_GRADIENT, --min-gradient MIN_GRADIENT